### Dependencies

In [1]:
import math
import numpy as np
import random
import pandas 
import plotly.express as px

### Functions

In [100]:
# return Gumbel's function value
def gumbel(x):
    return math.pow(math.e, x) * math.pow(math.e, -math.pow(math.e, x))

# return an array tuple of randomly sampled datapoints
def random_points(start, end, n_points):
    x_datapoints = sorted([start+(end-start)*random.random() for _ in range(n_points)])
    y_datapoints = list(map(gumbel, x_datapoints))
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array tuple of equally distanced datapoints
def equal_points(start, end, n_points):
    x_datapoints = [float(_) for _ in range(start, end+1, 1)]
    y_datapoints = list(map(gumbel, x_datapoints))
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array tuple of datapoints given by the Chebyshev points
def cheby_points(start, end, n_points):
    x_datapoints = sorted([start+(end-start)*((-math.cos((x-1)*math.pi/n_points)+1)/2) for x in range(n_points)])
    y_datapoints = list(map(gumbel, x_datapoints))  
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array of coefficients using Newton-Rhapson's divided differences
def nr_coeffts(x_datapoints, y_datapoints, n_points):
    a = y_datapoints.copy()
    for k in range(1, n_points):
        a[k:n_points] = (a[k:n_points] - a[k-1])/(x_datapoints[k:n_points] - x_datapoints[k-1])
    return a

# return the function value given by the interpolated function via the Newton-Rhapson method
def nr_evalfunct(coefficients, x_datapoints, x, degree):
    value = coefficients[degree]
    for k in range(1, degree+1):
        value = coefficients[degree-k] + (x - x_datapoints[degree-k])*value
    return value

# return the lagrange ith term
def lagrange_term(term, x_datapoints, i, x, degree):
    for j in range(degree+1):
        if j != i:
            term *= (x - x_datapoints[j]) / (x_datapoints[i] - x_datapoints[j])
            print(f"numerator {x - x_datapoints[j]}; denominator {x_datapoints[i] - x_datapoints[j]}; term {term}")
    return term

# return the function value given by the lagrange polynomial interpolation
def lagrange_evalfunct(x_datapoints, y_datapoints, x, degree) -> float:
    value = 0
    for i in range(degree+1):
        term = lagrange_term(y_datapoints[i], x_datapoints, i, x, degree)
        print(f"y dp: {y_datapoints[i]}")
        value += term
    return value

# return the reconstruction error of the newton rhapson interpolation
def poly_recon_error_nr(x_datapoints, y_datapoints, step_size, start, end, n_points):
    coeffts = nr_coeffts(x_datapoints, y_datapoints, n_points)
    error = 0
    for x in range(start*10, (end)*10+1, step_size):
       error += abs(gumbel(x/10) - nr_evalfunct(coeffts, x_datapoints, x/10, n_points-1))
    return error

# return the reconstruction error of the lagrange interpolation
def poly_recon_error_lagrange(x_datapoints, y_datapoints, step_size, start, end, n_points):
    error = 0
    for x in range(start*10, (end)*10+1, step_size):
       print(f"{lagrange_evalfunct(x_datapoints, y_datapoints, x/10, n_points-1)}")
    #    error += abs(gumbel(x/10) - lagrange_evalfunct(x_datapoints, y_datapoints, x/10, n_points-1))
    #    print("done", x, error)
    return error

# plot the polynomial interpolation
def plot_interp(x_datapoints, y_datapoints, step_size, start, end, n_points):
    coeffts = nr_coeffts(x_datapoints, y_datapoints, n_points)
    xdata = []
    ydata = []
    for x in range(start*10, (end)*10+1, step_size):
       xdata.append(x/10)
       ydata.append(nr_evalfunct(coeffts, x_datapoints, x/10, n_points-1))
    fig = px.scatter(x=xdata, y=ydata)
    fig.show()

def plot_interp_lag(x_datapoints, y_datapoints, step_size, start, end, n_points):
    xdata = []
    ydata = []
    for x in range(start*10, (end)*10+1, step_size):
       xdata.append(x/10)
       ydata.append(lagrange_evalfunct(x_datapoints, y_datapoints, x/10, n_points-1))
    fig = px.scatter(x=xdata, y=ydata)
    fig.show()

# plot the gumbel function
def plot_gumbel(step_size, start, end):
    xdata = []
    ydata = []
    for x in range(start*10, (end)*10+1, step_size):
       xdata.append(x/10)
       ydata.append(gumbel(x/10))
    fig = px.scatter(x=xdata, y=ydata)
    fig.show()

def radial_recon_error(x_datapoints, y_datapoints, step_size, start, end):
    ...

In [64]:
start = -5
end = 3
n_points = 9
step_size = 1   

random_x_datapoints, random_y_datapoints = random_points(start, end, n_points)
fig = px.scatter(x=random_x_datapoints, y=random_y_datapoints)
fig.show()

equal_x_datapoints, equal_y_datapoints = equal_points(start, end, n_points)
fig = px.scatter(x=equal_x_datapoints, y=equal_y_datapoints)
fig.show()

cheby_x_datapoints, cheby_y_datapoints = cheby_points(start, end, n_points)
fig = px.scatter(x=cheby_x_datapoints, y=cheby_y_datapoints)
fig.show()

In [75]:
print(cheby_x_datapoints)
print(random_x_datapoints)
print("------------")
print(cheby_y_datapoints)
print(random_y_datapoints)

[-2.5        -2.25877048 -2.25877048 -1.56417777 -0.5         0.80540729
  2.19459271  3.5         4.56417777]
[-1.64309795 -1.46490386 -0.66524224 -0.35588529  0.32695238  0.42556485
  1.00009721  2.15199031  2.46649626]
------------
[7.56161799e-02 9.41139170e-02 9.41139170e-02 1.69748428e-01
 3.30704299e-01 2.38782931e-01 1.13428630e-03 1.37458828e-13
 1.98167079e-40]
[1.59377770e-01 1.83414975e-01 3.07465865e-01 3.47691994e-01
 3.46530979e-01 3.31247383e-01 1.79344118e-01 1.58056705e-03
 9.00991094e-05]


In [101]:
print(poly_recon_error_nr(random_x_datapoints, random_y_datapoints, step_size, start, end, n_points))
print(poly_recon_error_nr(equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points))
print(poly_recon_error_lagrange(cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points))

1750.0594308872464
1.4152350082555476
numerator -2.7412295168563663; denominator -0.2412295168563663; term 0.8592700720989767
numerator -2.7412295168563663; denominator -0.2412295168563663; term 9.764379232212727
numerator -3.435822227524088; denominator -0.935822227524088; term 35.84940624115229
numerator -4.5; denominator -1.9999999999999996; term 80.66116404259266
numerator -5.805407289332278; denominator -3.3054072893322783; term 141.66814214096107
numerator -7.194592710667721; denominator -4.694592710667721; term 217.1103321626016
numerator -8.5; denominator -5.999999999999999; term 307.57297056368566
numerator -9.564177772475912; denominator -7.064177772475912; term 416.4225000029359
y dp: 0.07561617991742652
numerator -2.5; denominator 0.2412295168563663; term -0.9753565631723642
numerator -2.7412295168563663; denominator 0.0; term inf
numerator -3.435822227524088; denominator -0.6945927106677217; term inf
numerator -4.5; denominator -1.7587704831436333; term inf
numerator -5.80

/var/folders/7v/lh4dpmwx58b8f4ly3bvf48000000gn/T/ipykernel_7499/1457324209.py:41: RuntimeWarning:

divide by zero encountered in scalar divide

/var/folders/7v/lh4dpmwx58b8f4ly3bvf48000000gn/T/ipykernel_7499/1457324209.py:41: RuntimeWarning:

invalid value encountered in scalar multiply



In [61]:
# plot_interp(random_x_datapoints, random_y_datapoints, step_size, start, end, n_points)
plot_interp(equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points)
plot_interp_lag(cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points)
# plot_gumbel(step_size, start, end)

/var/folders/7v/lh4dpmwx58b8f4ly3bvf48000000gn/T/ipykernel_7499/2615123625.py:41: RuntimeWarning:

divide by zero encountered in scalar divide

/var/folders/7v/lh4dpmwx58b8f4ly3bvf48000000gn/T/ipykernel_7499/2615123625.py:41: RuntimeWarning:

invalid value encountered in scalar multiply

